# Load data and data analysis

In [ ]:
# TODO

# Setup generative model (open/closed model)

In [ ]:
from transformers import AutoTokenizer, OPTForCausalLM, pipeline

class GenerativeModel:
    def __init__(self, max_answer_length) -> None:
        self.generator = pipeline('text-generation', model="facebook/opt-1.3b")
        self.max_answer_length = max_answer_length

    def get_open_model_answer(self, question, context):
        prompt = f"CONTEXT:\n{context}\nQUESTION:\n{question}"
        # generate answer
        answer = self.generate_answer(prompt)
        # remove prompt from generated text
        answer = answer.removeprefix(prompt)
        return answer

    def get_closed_model_answer(self, question):
        prompt = question
        # generate answer
        answer = self.generate_answer(prompt)
        # remove prompt from generated text
        answer = answer.removeprefix(prompt)
        return answer
    
    def generate_answer(self, prompt):
        answer = self.generator(prompt, max_new_tokens=self.max_answer_length)[0]['generated_text']
        return answer

In [ ]:
generative_model = GenerativeModel(max_answer_length=42)

In [ ]:
# NOTE: test example from the squad dataset
# correct answer: The Review of Politics

context = """The Review of Politics was founded in 1939 by Gurian, modeled after German Catholic journals. It quickly emerged as part of an international Catholic intellectual revival, offering an alternative vision to positivist philosophy. For 44 years, the Review was edited by Gurian, Matthew Fitzsimons, Frederick Crosson, and Thomas Stritch. Intellectual leaders included Gurian, Jacques Maritain, Frank O'Malley, Leo Richard Ward, F. A. Hermens, and John U. Nef. It became a major forum for political ideas and modern political concerns, especially from a Catholic and scholastic tradition."""
question = "Gurian created what in 1939 at Notre Dame?"

print(generative_model.get_closed_model_answer(question))
print("#"*80)
print(generative_model.get_open_model_answer(question, context))

# Evaluation

In [ ]:
# TODO